In [2]:
import pandas as pd
import yfinance as yf
from yahoofinancials import YahooFinancials
import matplotlib as mpl
import matplotlib.pyplot as plt
from datetime import date, timedelta, datetime
import sqlite3    
import re   
import requests   
from bs4 import BeautifulSoup   
import plotly.graph_objects as go
#import win32com.client as win32
import plotly.io as pio
from plotly.subplots import make_subplots

In [3]:
pd.set_option('display.max_rows', None)

#### Funções Data ( não são usadas para as funções SQL)

In [4]:
# Dado o offset um número inteiro, retorna a data de agora menos o offset.

def day_n (offset):         
    
    return datetime.today()-timedelta(days = offset)         

In [5]:
def sunday(offset, day):    
    
    if day.weekday()==6:     
        day = day_n(offset+2)         
        if day.strftime("%d-%m-%Y") == holiday(offset, day, dataf).strftime("%d-%m-%Y"):         
            return day         
        else:              return holiday(offset, day, dataf)         
    
    return day   

####  Caso seja domingo essa função retorna o última dia antes de começar o final de semana
####  seja ele feriado ou não.  
####  Caso não seja domingo ele retorna o mesmo dia que foi passado como argumento pela função, sem alterá-lo.   

In [6]:
def saturday(offset, day):
    
    if day.weekday()==5:     
        day = day_n(offset+1)         
        if day.strftime("%d-%m-%Y") == holiday(offset, day, dataf).strftime("%d-%m-%Y"):         
            return day         
        else:              return holiday(offset, day, dataf)     
    return day   

####  Caso seja sabado essa função retorna o última dia antes de começar o final de semana, 
####  seja ele feriado ou não.  
####  Caso não seja sábado ele retorna o mesmo dia que foi passado como argumento pela função, sem alterá-lo.   

In [7]:
def holiday(offset, day, dataf):     
    
    for i in dataf:     
        if (i[8:]+"-"+i[5:7]+"-"+i[0:4])==day.strftime("%d-%m-%Y"):             
            return holiday(offset+1, day_n(offset+1), dataf)         
    
            
    if day.strftime("%d-%m-%Y") == sunday(offset, day).strftime("%d-%m-%Y"):         
    
        
        if day.strftime("%d-%m-%Y") == saturday(offset, day).strftime("%d-%m-%Y"):             
            
            return day         
        else:             
            return saturday(offset, day)         
    else:         
        return sunday(offset, day)         

In [8]:
##### Descobrindo o D-1 #####

def find_last_workday( offset, day, dataf): 
    
    if day.strftime("%d-%m-%Y") == holiday(offset, day, dataf).strftime("%d-%m-%Y"):         
        
        if day.strftime("%d-%m-%Y") == sunday(offset, day).strftime("%d-%m-%Y"):             
            
            if day.strftime("%d-%m-%Y") == saturday(offset, day).strftime("%d-%m-%Y"):                 
                return day                 
                
            else:                     
                return saturday(offset, day)         
        else:                 
            return sunday(offset, day)         
    else:         
        return holiday(offset, day, dataf)          

#### Vetor de feriados ( não são usadas para as funções SQL)

In [9]:
def holidays():
    
    feriados = pd.read_csv(r"C:\Users\psilv\Python\feriados.csv")   
 
 
    data = feriados['data'].tolist()   
 
    ########### Cria vetor de feriados 
    
    dataf=[] 
    
    for i in data:   
        
        dataf.append(i.strip()[-4:] + "-" + ("0"+ i.strip()[0] if i.find("/")==1 else i.strip()[0:i.find("/")]) + "-" + ("0" + re.search("/(.*?)/", i.strip()).group(1) if len(re.search("/(.*?)/", i.strip()).group(1))==1 else re.search("/(.*?)/", i.strip()).group(1)))
        
    return dataf     

#### Vetor de feriados ( é usada para as funções SQL)

In [10]:
def past_n_days(n):     
    day = datetime.today()-timedelta(days = n)     
    print("O último dia corrido é: " +  day.strftime("%d-%m-%Y"))     
    return day   

####  Função auxiliar para criar Vetor de dias úteis ( é usada para as funções SQL)

In [249]:
##### Date Format input  YY-MM-dd,  run_days são números inteiros

##### O input de data dentro da função pandas.date_range deve ser no formato YY-MM-dd

def date_array (last_day, run_days):

   
    Datas = pd.date_range(end = last_day.strftime("%Y-%m-%d"), periods = run_days).to_pydatetime().tolist()

    
    datas=[] 

    
    for k in range(len(Datas)):

        datas.append(Datas[len(Datas) - k -1])
 
    return datas

In [250]:
date_array(datetime.today(),5)

[datetime.datetime(2023, 6, 13, 0, 0),
 datetime.datetime(2023, 6, 12, 0, 0),
 datetime.datetime(2023, 6, 11, 0, 0),
 datetime.datetime(2023, 6, 10, 0, 0),
 datetime.datetime(2023, 6, 9, 0, 0)]

#### Função iterativa para criar Vetor de dias úteis. ( não são usadas para as funções SQL)

In [223]:
#### Função para obter um vetor de dias úteis de um tamanho específico, por ser iterativa é custosa computacionalmente

#### Date format input dd-MM-YY, length é número inteiro


def work_days_array_length(last_day, length):


    i=length

    while len(date_array(last_day, i))!= length:

        if len(date_array(last_day, i))==length:

            break

        i+=1

    return date_array(last_day, i)

#### Função  Vetor de dias úteis. ( é usada para as funções SQL)

In [263]:
####### Criando vetor desde o ultimo dia do banco até o D-1

####### Start é a mais antiga, e End a mais recente

####### Date Format input  YY-MM-dd (SQL)


def  work_days_array_btw_dates(start_date, end_date):
    

    end = end_date
    #print(end)

    start = start_date
    print(start)

    
    dif = end-start
    
    #print(dif)
    

    dif = int(abs(dif.days))
    
    #print(dif)
    
    
    print("Números de dias a serem atualizados:  " + str(dif))
    print("\n")
    
    #### No caso start_date=end_date é preciso desse if 
    
    if dif == 0:
        dif=2
            


    dates = date_array(end_date, 2*dif)
    
    print(dates)
    print(start_date)

    aux=0
    

    for i in dates:

        if i==start_date:

            aux = dates.index(i)
                 
    
    return dates[:aux+1]

In [258]:
(datetime.today()-timedelta(days = 5)).date()

datetime.date(2023, 6, 9)

In [ ]:
## DATETIME na funcao date_array ta com tempo zerado, ja na funcao work_days_array_btw_dates 
## esta no formato date sem tempo, buscar uma forma de consertar, pois é preciso inserir no banco com o tempo

In [246]:
datetime.today().date()

datetime.date(2023, 6, 13)

In [269]:
work_days_array_btw_dates(datetime(2023,6,5), datetime(2023,6,13))

2023-06-05 00:00:00
Números de dias a serem atualizados:  8


[datetime.datetime(2023, 6, 13, 0, 0), datetime.datetime(2023, 6, 12, 0, 0), datetime.datetime(2023, 6, 11, 0, 0), datetime.datetime(2023, 6, 10, 0, 0), datetime.datetime(2023, 6, 9, 0, 0), datetime.datetime(2023, 6, 8, 0, 0), datetime.datetime(2023, 6, 7, 0, 0), datetime.datetime(2023, 6, 6, 0, 0), datetime.datetime(2023, 6, 5, 0, 0), datetime.datetime(2023, 6, 4, 0, 0), datetime.datetime(2023, 6, 3, 0, 0), datetime.datetime(2023, 6, 2, 0, 0), datetime.datetime(2023, 6, 1, 0, 0), datetime.datetime(2023, 5, 31, 0, 0), datetime.datetime(2023, 5, 30, 0, 0), datetime.datetime(2023, 5, 29, 0, 0)]
2023-06-05 00:00:00


[datetime.datetime(2023, 6, 13, 0, 0),
 datetime.datetime(2023, 6, 12, 0, 0),
 datetime.datetime(2023, 6, 11, 0, 0),
 datetime.datetime(2023, 6, 10, 0, 0),
 datetime.datetime(2023, 6, 9, 0, 0),
 datetime.datetime(2023, 6, 8, 0, 0),
 datetime.datetime(2023, 6, 7, 0, 0),
 datetime.datetime(2023, 6, 6, 0, 0),
 datetime.datetime(2023, 6, 5, 0, 0)]

#### Função para deletar linha específica

In [14]:
def delete_row_sql(table_name, column_name, conn, desired_info):
    
    
    query = ("""DELETE""" + 
             """ FROM """ + table_name +
             """ WHERE """ + column_name +
             """ = '""" + desired_info +
             """'; """)
    
    conn.execute(query)
    
    print("Deleted as Follows:")
    print("\n")
    print(query)

#### Função para obter um dataframe a partir de uma query do banco de dados

In [84]:
### Criar função para obter um dataframe entre datas específicas a partir de uma query em SQL

def df_from_sql_query(table_name, date_column_name, conn, start_date, end_date):
    
    
    query = ("""SELECT * """ + 
             """ FROM """ + table_name +
             """ WHERE """ + date_column_name +
             """ BETWEEN strftime('%Y-%m-%d %H:%M:%S', '""" + start_date +
             """') AND strftime('%Y-%m-%d %H:%M:%S', '""" + end_date +
             """') ORDER BY """ + date_column_name + """; """)
    
    #print(query)

    df = pd.read_sql_query(query, conn)
    
    print(df.dtypes)
    
    
    return df   

In [85]:
df_from_sql_query('ETH', "Date", conn, "2022-01-28", "2022-07-25")

Date         datetime64[ns]
Open                float64
High                float64
Low                 float64
Close               float64
Adj Close           float64
Volume                int64
dtype: object


,Date,Open,High,Low,Close,Adj Close,Volume
0,2022-01-28,2421.646973,2548.778564,2367.833740,2547.092041,2547.092041,14666227351
1,2022-01-29,2546.590576,2627.609375,2526.989746,2597.084717,2597.084717,11172062661
2,2022-01-30,2598.564941,2631.408447,2550.460938,2603.466553,2603.466553,9501221177
3,2022-01-31,2603.263428,2697.735352,2489.072266,2688.278809,2688.278809,13778234614
4,2022-02-01,2687.898926,2802.315430,2682.621826,2792.117188,2792.117188,13194846235
5,2022-02-02,2791.958984,2802.212158,2630.120361,2682.854004,2682.854004,13876301217
6,2022-02-03,2682.226074,2712.482666,2587.783447,2679.162598,2679.162598,12755505065
7,2022-02-04,2681.057617,2983.586914,2675.443848,2983.586914,2983.586914,18987223729
8,2022-02-05,2984.446045,3054.130127,2966.781006,3014.648193,3014.648193,13102093957
9,2022-02-06,3014.959717,3061.261230,2965.429932,3057.476074,3057.476074,9466018022


#### Inserindo novas linhas no banco de dados

In [17]:
##### Tornar essa função genérica para qualquer dataframe

def insert_n_lines_database(cursor, df, table_name, date_column_name):
    
           
    print("\n")     
    print("Os dados antes da atualização eram:")   
    print("\n")  
    
    for i in get_last_n_rows(cursor, table_name, 5, date_column_name):   
        print(i)             
 
    print("\n")     
   
    insert_n_rows(cursor, df, table_name)             
       
    print("\n")   
    print("Os novos dados são:")   
    print("\n")     
                   
    for i in get_last_n_rows(cursor, table_name, 5, date_column_name):   
        print(i)         
    print("\n")             
            

#### Atualizando as bases de dados

In [274]:
def updating_database(cursor, date_column_name):      
    
    for i in assets:
        
        print('\n')
        
        day = past_n_days(1)
    
        #day = day.strftime("%Y-%m-%d")
        
        ###### Verificando a última data atualizada no banco de dados   
        last_date_db = get_last_db_date(cursor, date_column_name, i)
        
        

        if last_date_db.strftime("%Y-%m-%d") == day.strftime("%Y-%m-%d"):         
            print("Os dados da cripto " + str(i) + " estão Atualizados!!")         
        else:         
            print("\n")     
            print("O último dia atualizado no banco de dados é: " + last_date_db.strftime("%Y-%m-%d"))   
            print("\n")         

            #### Vetor de datas   
            dates = work_days_array_btw_dates(last_date_db, day)[:-1]   
            print("Datas a serem atualizadas no banco de dados:")         
            print("\n")         

            for j in range(len(dates)):             
                print(dates[j])             


            print("\n")
            print("Downloading from Yahoo Finance API ....")         
            print("\n")         

            #### Dataframe com os resultados
            
            print("Moeda Atualizada: " + assets[i])
            
            print(last_date_db.date())
            print(day.date())
            print((day-last_date_db).days)
            
            print(dates)
            print(dates[0])
            print(dates[0].strftime("%Y-%m-%d"))
            print(dates[-1:][0].strftime("%Y-%m-%d"))
            
    

            df = get_data_btw_date(assets[i],dates[-1:][0].strftime("%Y-%m-%d"), dates[0].strftime("%Y-%m-%d"))     


            #### Inserindo novos dados           
            insert_n_lines_database(cursor, df, i, date_column_name)  

            ##### Adicionar o insert lines dentro da função Web Scrap para atualizar o banco de dados em cada iteração e não
            ##### correr o risco de falhar o web scraping e perder os dados
    
    

#### Função para captar os dados da API do Yahoo Finance

In [77]:
#### Nome da moeda, exemplo: Ethereum é ETH-USD
#### Data format YY-MM-DD

def get_data_btw_date(crypto_name, start_date, end_date):
    
    crypto_df = yf.download(crypto_name, start= start_date, end= end_date)
    
    crypto_df = crypto_df.reset_index(level=0)
    
    #print(crypto_df.dtypes)
    #print(crypto_df)
    
    ## Agora o banco de dados recebe a coluna de data como tipo data, ateriormente era texto. 
    ##As operações abaixo transformavam para texto.
    
    #crypto_df['Date'] = pd.to_datetime(crypto_df['Date'], format='%Y-%m-%d %H:%M:%S')
    
    #crypto_df['Date'] = crypto_df['Date'].dt.strftime('%Y-%m-%d %H:%M:%S')

    #print(crypto_df.dtypes)
    #print(crypto_df)

    return crypto_df

In [145]:
a = get_data_btw_date('BTC-USD', '2023-01-12', '2023-06-05')

[*********************100%***********************]  1 of 1 completed


In [76]:
print(a.dtypes)

Date         datetime64[ns]
Open                float64
High                float64
Low                 float64
Close               float64
Adj Close           float64
Volume                int64
dtype: object


#### Funções SQL

In [22]:
## Insert crypto name in lower case

In [23]:
#### index can be False or True, to me worked as False

def create_table(df, name_desired, conn, index):
    
        df.to_sql(name_desired, conn, if_exists='fail', index=index)   
    


In [148]:
create_table(a,"BTC",conn, index=False)

In [24]:
def rename_table(table_name, new_table_name, cursor):
    
    try:
        
        for i in get_names_tables(cursor):
        
            if new_table_name == i[0]:
                
                print("O Nome já está em uso!")
            
                raise Exception

        cursor.execute("""ALTER TABLE """ +  table_name + """ RENAME TO """ + new_table_name + """ ;""")

    except Exception:

        print("Não foi possível alterar o nome da tabela!")
        

In [25]:
def show_table(table_name, cursor):     
    
    c=cursor.execute("""SELECT * FROM """ + table_name ).fetchall()     
    for i in c:      
        print(i)     
    return c     

In [26]:
def delete_table(name, cursor):   
    
    cursor.execute("""DROP TABLE """ +  name ).fetchall()   

In [27]:
def last_database_day(crypto_name):

    sql_str = '''SELECT Date FROM ''' + crypto_name + ''' ORDER BY Date DESC LIMIT 1;'''

    c.execute(sql_str)

    date = c.fetchone()

    return date[0][:10]
    

In [28]:
def get_names_tables(cursor):     
    
    cursor.execute('''SELECT name FROM sqlite_master WHERE type='table';''')   
    
    return(cursor.fetchall())

In [29]:
def count_rows_table(cursor, table_name): 
    
    c = cursor.execute("""SELECT COUNT(*) FROM """ + table_name).fetchall()     
    return c[0][0]   

In [30]:
def get_columns_names(cursor, table_name): 
    
    c = cursor.execute("""PRAGMA table_info(""" + table_name + """);  """ ).fetchall()     
    col_names = []     
    for i in c:         
            col_names.append(i[1])
            
    return col_names   

In [31]:
def get_last_n_rows(cursor, table_name, n_rows, column_name):
    
    c = cursor.execute("""SELECT * FROM """ 
                       + table_name + 
                       """ ORDER BY """ +column_name + 
                       """ DESC LIMIT """+ str(n_rows) + 
                       """ ; """  ).fetchall()     
#     for i in c:      
#         print(i)


    
    return c     

In [275]:
def insert_n_rows(cursor, df, table_name):     
 
   
    new_values = []   
    
    df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d %H:%M:%S')
    
    df['Date'] = df['Date'].dt.strftime('%Y-%m-%d %H:%M:%S')
    
    for i in range(len(df)):
        
        #print(df.iloc[i])
        
        new_values.append(tuple(df.iloc[i]))
        
    
    s=""     
    
    for i in new_values:   
        
        s+=str(i) + ","   
        
    
    s=s[:-1] 

    #print(s)
    print("")
    
    
    cursor.execute("""INSERT INTO """ + table_name + """ VALUES """ + s + """ ; """  )     
    

In [33]:
def distinct_values(table_name, column_name, cursor):
    
    c = cursor.execute("""SELECT """ + column_name + """, COUNT(*) """ + """ FROM """ + table_name + """ GROUP BY """ +  column_name + """ ; """  ).fetchall()     
    
    
    for i in c:
        
        print(i)
    
       

In [34]:
def change_columns_names(cursor, table_name, column_old_name, column_new_name):   
 
    print("Old Columns Names: " + str(get_columns_names(cursor, table_name)) )     
    c = cursor.execute("""ALTER TABLE """ + table_name +                                 """ RENAME COLUMN """ + column_old_name +                                 """ TO """+column_new_name+                                 """ ; """  ).fetchall()     
    print("New Columns Names: " + str(get_columns_names(cursor, table_name)) )     
 

In [35]:
def get_last_db_date(cursor,column_name, table_name):  
    
    c=cursor.execute("""SELECT """ + column_name + 
                     """ FROM """ + table_name + 
                     """ ORDER BY """ + column_name + 
                     """ DESC LIMIT 1 ; """).fetchall()     
    return c[0][0]   

In [130]:
get_last_db_date(c,"Date", "UNI1")

datetime.datetime(2022, 10, 9, 0, 0)

In [36]:
def database_info(cursor, table_name, date_column_name):
    
        
    
    print("O banco de dados contém as seguintes tabelas: ")
    
    print("\n")
    
    for i in get_names_tables(cursor):
        print(i[0])
    
    print("\n")
    
    print("O nome da tabela solicitada é :" + table_name)
    
    print("\n")
    
    print("O nome das colunas são: ")
    
    print("\n")
    
    print(get_columns_names(cursor, table_name))
    
    print("\n")
    
    print("O número total de linhas é: " + str(count_rows_table(cursor, table_name)))
    
    print("\n")
    
    print("A data mais recente dos dados é: " + str(get_last_db_date(cursor,date_column_name, table_name)))
    
    print("\n")
    
    print("Todas as datas distintas e quantas linhas cada uma possui são: ")
    
    print("\n")
    
    print(distinct_values(table_name, date_column_name, cursor)) 
    
    

In [37]:
def get_new_crypto_in_yf(crypto_name, table_title, conn, end_date, index=False):
    
 
    df = get_data_btw_date(crypto_name, '2000-01-01', end_date)
        
    #### Data escolhida é bem antes do surgimento das criptomoedas no planeta terra
        
        
    create_table(df, table_title, conn, index)
    

In [38]:
###### updating entire database for all assets ##########

def insert_new_criptos_in_db(assets):
    
    day = past_n_days(1)
    
    day = day.strftime("%Y-%m-%d")

    for i in assets:
        
        try:

            get_new_crypto_in_yf(assets[i], i, conn, day, index=False)
            
        
        except Exception:
            
            print(str(assets[i]+ " já  está no banco de dados!"))
            
            continue

In [140]:
assets = { 'BTC': 'BTC-USD', 
           'ETH': 'ETH-USD', 
           'USDT':'USDT-USD', 
           'USDC':'USDC-USD', 
           'BNB':'BNB-USD',
           'BUSD':'BUSD-USD',
           'XRP': 'XRP-USD',
           'ADA': 'ADA-USD',
           'SOL': 'SOL-USD',
           'DOGE':'DOGE-USD',
           'DOT': 'DOT-USD',
           'HEX': 'HEX-USD',
           'SHIB': 'SHIB-USD',
           'DAI': 'DAI-USD',
           'WTRX': 'WTRX-USD',
           'AVAX':'AVAX-USD',
           'MATIC': 'MATIC-USD',
           'TRX': 'TRX-USD',
           'STETH': 'STETH-USD',
           'WBTC': 'WBTC-USD',
           'LEO': 'LEO-USD',
           'ETC': 'ETC-USD',
           'LTC': 'LTC-USD'
         }

#### Rodando

In [283]:
## These detect_types allow us to read date columns as date type that was stored as a string by sqlite3

conn = sqlite3.connect('crypto_data', detect_types=sqlite3.PARSE_DECLTYPES|sqlite3.PARSE_COLNAMES)
c = conn.cursor()


In [281]:
c.close()


ProgrammingError: Cannot operate on a closed database.

In [284]:
updating_database(c, "Date")



O último dia corrido é: 13-06-2023


O último dia atualizado no banco de dados é: 2023-06-05


2023-06-05 00:00:00
Números de dias a serem atualizados:  8


[datetime.datetime(2023, 6, 13, 0, 0), datetime.datetime(2023, 6, 12, 0, 0), datetime.datetime(2023, 6, 11, 0, 0), datetime.datetime(2023, 6, 10, 0, 0), datetime.datetime(2023, 6, 9, 0, 0), datetime.datetime(2023, 6, 8, 0, 0), datetime.datetime(2023, 6, 7, 0, 0), datetime.datetime(2023, 6, 6, 0, 0), datetime.datetime(2023, 6, 5, 0, 0), datetime.datetime(2023, 6, 4, 0, 0), datetime.datetime(2023, 6, 3, 0, 0), datetime.datetime(2023, 6, 2, 0, 0), datetime.datetime(2023, 6, 1, 0, 0), datetime.datetime(2023, 5, 31, 0, 0), datetime.datetime(2023, 5, 30, 0, 0), datetime.datetime(2023, 5, 29, 0, 0)]
2023-06-05 00:00:00
Datas a serem atualizadas no banco de dados:


2023-06-13 00:00:00
2023-06-12 00:00:00
2023-06-11 00:00:00
2023-06-10 00:00:00
2023-06-09 00:00:00
2023-06-08 00:00:00
2023-06-07 00:00:00
2023-06-06 00:00:00




Moeda Atua

[*********************100%***********************]  1 of 1 completed


Os dados antes da atualização eram:


(datetime.datetime(2023, 6, 12, 0, 0), 235.40391540527344, 238.4732208251953, 222.0713348388672, 231.0514373779297, 231.0514373779297, 1014433334)
(datetime.datetime(2023, 6, 11, 0, 0), 239.08741760253906, 239.3482666015625, 234.43069458007812, 235.4387969970703, 235.4387969970703, 376213808)
(datetime.datetime(2023, 6, 10, 0, 0), 260.69866943359375, 260.7160949707031, 233.169189453125, 239.08908081054688, 239.08908081054688, 1068424942)
(datetime.datetime(2023, 6, 9, 0, 0), 262.1507873535156, 263.9214172363281, 256.5093994140625, 260.7160949707031, 260.7160949707031, 491429998)
(datetime.datetime(2023, 6, 8, 0, 0), 259.4009094238281, 266.8072204589844, 257.7645263671875, 262.14849853515625, 262.14849853515625, 562863852)





Os novos dados são:


(datetime.datetime(2023, 6, 13, 0, 0), 231.07504272460935, 245.64219665527344, 229.3116912841797, 243.8905334472656, 243.89053344726

[*********************100%***********************]  1 of 1 completed


Os dados antes da atualização eram:


(datetime.datetime(2023, 6, 12, 0, 0), 15.557231903076172, 15.606237411499023, 14.817190170288086, 15.196707725524902, 15.196707725524902, 384209938)
(datetime.datetime(2023, 6, 11, 0, 0), 15.654450416564941, 16.115692138671875, 15.198493003845215, 15.558775901794434, 15.558775901794434, 446176848)
(datetime.datetime(2023, 6, 10, 0, 0), 17.3813533782959, 17.39295768737793, 13.379448890686035, 15.65610122680664, 15.65610122680664, 1230339924)
(datetime.datetime(2023, 6, 9, 0, 0), 18.908403396606445, 19.299476623535156, 17.161293029785156, 17.37969970703125, 17.37969970703125, 463344687)
(datetime.datetime(2023, 6, 8, 0, 0), 18.63564682006836, 19.0777645111084, 18.294736862182617, 18.909475326538086, 18.909475326538086, 258460307)





Os novos dados são:


(datetime.datetime(2023, 6, 13, 0, 0), 15.196287155151367, 15.873085975646973, 14.816584587097168, 15.018193244934082, 15.018

[*********************100%***********************]  1 of 1 completed


Os dados antes da atualização eram:


(datetime.datetime(2023, 6, 12, 0, 0), 7.000000096013537e-06, 7.000000096013537e-06, 6.000000212225132e-06, 7.000000096013537e-06, 7.000000096013537e-06, 100135834)
(datetime.datetime(2023, 6, 11, 0, 0), 7.000000096013537e-06, 7.000000096013537e-06, 7.000000096013537e-06, 7.000000096013537e-06, 7.000000096013537e-06, 111177162)
(datetime.datetime(2023, 6, 10, 0, 0), 7.999999979801942e-06, 7.999999979801942e-06, 6.000000212225132e-06, 7.000000096013537e-06, 7.000000096013537e-06, 402522019)
(datetime.datetime(2023, 6, 9, 0, 0), 7.999999979801942e-06, 7.999999979801942e-06, 7.999999979801942e-06, 7.999999979801942e-06, 7.999999979801942e-06, 78014639)
(datetime.datetime(2023, 6, 8, 0, 0), 7.999999979801942e-06, 7.999999979801942e-06, 7.999999979801942e-06, 7.999999979801942e-06, 7.999999979801942e-06, 69733322)





Os novos dados são:


(datetime.datetime(2023, 6, 13, 0, 0), 7.00

[*********************100%***********************]  1 of 1 completed


Os dados antes da atualização eram:


(datetime.datetime(2023, 6, 12, 0, 0), 0.6350859999656677, 0.6513260006904602, 0.6212430000305176, 0.6430230140686035, 0.6430230140686035, 437837502)
(datetime.datetime(2023, 6, 11, 0, 0), 0.6118559837341309, 0.6575210094451904, 0.5951200127601624, 0.6350730061531067, 0.6350730061531067, 561201492)
(datetime.datetime(2023, 6, 10, 0, 0), 0.7276399731636047, 0.7276399731636047, 0.559319019317627, 0.6119580268859863, 0.6119580268859863, 1177044809)
(datetime.datetime(2023, 6, 9, 0, 0), 0.7814450263977051, 0.7878739833831787, 0.7241910099983215, 0.727711021900177, 0.727711021900177, 414939391)
(datetime.datetime(2023, 6, 8, 0, 0), 0.7653229832649231, 0.7843930125236511, 0.7578309774398804, 0.7813959717750549, 0.7813959717750549, 276953404)





Os novos dados são:


(datetime.datetime(2023, 6, 13, 0, 0), 0.6430590152740479, 0.6729679703712463, 0.627938985824585, 0.6443089842796326, 

[*********************100%***********************]  1 of 1 completed


Os dados antes da atualização eram:


(datetime.datetime(2023, 6, 12, 0, 0), 3.5504679679870605, 3.55846905708313, 3.493980884552002, 3.506972074508667, 3.506972074508667, 794368)
(datetime.datetime(2023, 6, 11, 0, 0), 3.50335693359375, 3.555026054382324, 3.4761390686035156, 3.5508110523223877, 3.5508110523223877, 595354)
(datetime.datetime(2023, 6, 10, 0, 0), 3.5020248889923096, 3.5807409286499023, 3.479001998901367, 3.5031590461730957, 3.5031590461730957, 843174)
(datetime.datetime(2023, 6, 9, 0, 0), 3.5374739170074463, 3.5699350833892822, 3.490330934524536, 3.5019469261169434, 3.5019469261169434, 763793)
(datetime.datetime(2023, 6, 8, 0, 0), 3.515294075012207, 3.5389750003814697, 3.5001609325408936, 3.5366909503936768, 3.5366909503936768, 543841)





Os novos dados são:


(datetime.datetime(2023, 6, 13, 0, 0), 3.507162094116211, 3.5187599658966064, 3.4794960021972656, 3.493371963500977, 3.493371963500977, 606299

In [285]:
database_info(c, 'ADA','Date')

O banco de dados contém as seguintes tabelas: 


ETH
USDT
USDC
BNB
BUSD
XRP
ADA
SOL
DOGE
DOT
HEX
SHIB
DAI
WTRX
AVAX
MATIC
TRX
STETH
WBTC
LEO
ETC
LTC
BTC


O nome da tabela solicitada é :BTC


O nome das colunas são: 


['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']


O número total de linhas é: 153


A data mais recente dos dados é: 2023-06-13 00:00:00


Todas as datas distintas e quantas linhas cada uma possui são: 


(datetime.datetime(2023, 1, 12, 0, 0), 1)
(datetime.datetime(2023, 1, 13, 0, 0), 1)
(datetime.datetime(2023, 1, 14, 0, 0), 1)
(datetime.datetime(2023, 1, 15, 0, 0), 1)
(datetime.datetime(2023, 1, 16, 0, 0), 1)
(datetime.datetime(2023, 1, 17, 0, 0), 1)
(datetime.datetime(2023, 1, 18, 0, 0), 1)
(datetime.datetime(2023, 1, 19, 0, 0), 1)
(datetime.datetime(2023, 1, 20, 0, 0), 1)
(datetime.datetime(2023, 1, 21, 0, 0), 1)
(datetime.datetime(2023, 1, 22, 0, 0), 1)
(datetime.datetime(2023, 1, 23, 0, 0), 1)
(datetime.datetime(2023, 1, 24, 0, 0), 1)
(datetime.date

In [146]:
delete_table('BTC', c)

In [87]:
insert_new_criptos_in_db(assets)

O último dia corrido é: 12-06-2023
[*********************100%***********************]  1 of 1 completed
BTC-USD já  está no banco de dados!
[*********************100%***********************]  1 of 1 completed
ETH-USD já  está no banco de dados!
[*********************100%***********************]  1 of 1 completed
USDT-USD já  está no banco de dados!
[*********************100%***********************]  1 of 1 completed
USDC-USD já  está no banco de dados!
[*********************100%***********************]  1 of 1 completed
BNB-USD já  está no banco de dados!
[*********************100%***********************]  1 of 1 completed
BUSD-USD já  está no banco de dados!
[*********************100%***********************]  1 of 1 completed
XRP-USD já  está no banco de dados!
[*********************100%***********************]  1 of 1 completed
ADA-USD já  está no banco de dados!
[*********************100%***********************]  1 of 1 completed
SOL-USD já  está no banco de dados!
[****************

In [142]:
df_from_sql_query('ETH', "Date", conn, "2022-01-28", "2022-07-25")

Date         datetime64[ns]
Open                float64
High                float64
Low                 float64
Close               float64
Adj Close           float64
Volume                int64
dtype: object


,Date,Open,High,Low,Close,Adj Close,Volume
0,2022-01-28,2421.646973,2548.778564,2367.833740,2547.092041,2547.092041,14666227351
1,2022-01-29,2546.590576,2627.609375,2526.989746,2597.084717,2597.084717,11172062661
2,2022-01-30,2598.564941,2631.408447,2550.460938,2603.466553,2603.466553,9501221177
3,2022-01-31,2603.263428,2697.735352,2489.072266,2688.278809,2688.278809,13778234614
4,2022-02-01,2687.898926,2802.315430,2682.621826,2792.117188,2792.117188,13194846235
5,2022-02-02,2791.958984,2802.212158,2630.120361,2682.854004,2682.854004,13876301217
6,2022-02-03,2682.226074,2712.482666,2587.783447,2679.162598,2679.162598,12755505065
7,2022-02-04,2681.057617,2983.586914,2675.443848,2983.586914,2983.586914,18987223729
8,2022-02-05,2984.446045,3054.130127,2966.781006,3014.648193,3014.648193,13102093957
9,2022-02-06,3014.959717,3061.261230,2965.429932,3057.476074,3057.476074,9466018022


In [172]:
database_info(c, 'BTC','Date')

O banco de dados contém as seguintes tabelas: 


ETH
USDT
USDC
BNB
BUSD
XRP
ADA
SOL
DOGE
DOT
HEX
SHIB
DAI
WTRX
AVAX
MATIC
TRX
STETH
WBTC
LEO
ETC
LTC
BTC


O nome da tabela solicitada é :BTC


O nome das colunas são: 


['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']


O número total de linhas é: 145


A data mais recente dos dados é: 2023-06-05 00:00:00


Todas as datas distintas e quantas linhas cada uma possui são: 


(datetime.datetime(2023, 1, 12, 0, 0), 1)
(datetime.datetime(2023, 1, 13, 0, 0), 1)
(datetime.datetime(2023, 1, 14, 0, 0), 1)
(datetime.datetime(2023, 1, 15, 0, 0), 1)
(datetime.datetime(2023, 1, 16, 0, 0), 1)
(datetime.datetime(2023, 1, 17, 0, 0), 1)
(datetime.datetime(2023, 1, 18, 0, 0), 1)
(datetime.datetime(2023, 1, 19, 0, 0), 1)
(datetime.datetime(2023, 1, 20, 0, 0), 1)
(datetime.datetime(2023, 1, 21, 0, 0), 1)
(datetime.datetime(2023, 1, 22, 0, 0), 1)
(datetime.datetime(2023, 1, 23, 0, 0), 1)
(datetime.datetime(2023, 1, 24, 0, 0), 1)
(datetime.date

In [80]:
for i in assets:
    print(i)
    delete_table(i, c)

BTC
ETH
USDT
USDC
BNB
BUSD
XRP
ADA
SOL
DOGE
DOT
HEX
SHIB
DAI
WTRX
AVAX
MATIC
TRX
STETH
WBTC
UNI1
LEO
ETC
LTC


In [ ]:
def plot(title,crypto, conn, startDate, endDate):
    
    
    df=df_from_sql_query(crypto, "Date", conn, startDate, endDate)
    
    df['Date']= pd.to_datetime(df['Date'])
    
    df=df.drop(columns=['Adj Close'])
    
#     eth_df.set_index('Date')

    fig = go.Figure(data=[go.Candlestick(x=df['Date'],
                    open=df['Open'],
                    high=df['High'],
                    low=df['Low'],
                    close=df['Close'])])

    fig.update_layout(xaxis_rangeslider_visible=False)

    fig.update_layout(
        title= title,
        yaxis_title='Price',

    )

    fig.write_html(title + ".html")
    fig.write_html(title + ".html")

    fig.show()

In [ ]:
plot('ETHER-USD','ETH', conn, '2022-08-01', '2022-09-14')

In [ ]:
plot('BTC-USD','BTC', conn, '2022-08-01', '2022-09-14')